In [1]:
import pandas as pd
df = pd.read_csv('inference.csv')

In [17]:
df_cleaned = df
df_cleaned.dtypes

tree_id               int64
block_id              int64
created_at           object
tree_dbh              int64
stump_diam            int64
curb_loc             object
status               object
health               object
spc_latin            object
spc_common           object
steward              object
guards              float64
sidewalk             object
user_type            object
problems             object
root_stone           object
root_grate           object
root_other           object
trunk_wire           object
trnk_light           object
trnk_other           object
brch_light           object
brch_shoe            object
brch_other           object
address              object
postcode              int64
zip_city             object
community board       int64
borocode              int64
borough              object
cncldist              int64
st_assem              int64
st_senate             int64
nta                  object
nta_name             object
boro_ct             

In [3]:
df_cleaned['council district'] = df_cleaned.groupby(['postcode', 'borough'])['council district'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x.median()))
df_cleaned['census tract'] = df_cleaned.groupby(['postcode', 'borough'])['census tract'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x.median()))

In [4]:
df_cleaned = df_cleaned.dropna(subset=['health'])
df_cleaned = df_cleaned.dropna(subset=['spc_latin'])
df_cleaned = df_cleaned.dropna(subset=['spc_common'])
df_cleaned = df_cleaned.dropna(subset=['sidewalk'])
df_cleaned = df_cleaned.dropna(subset=['council district'])
df_cleaned = df_cleaned.dropna(subset=['census tract'])

In [5]:
df_cleaned['steward'] = df_cleaned['steward'].fillna('None')
df_cleaned['guards'] = df_cleaned['guards'].fillna('None')

In [6]:
df_cleaned = df_cleaned.drop(columns=['block_id', 'tree_id', 'created_at', 'latitude', 'longitude', 'x_sp', 'y_sp', 'boro_ct', 'nta', 'nta_name', 'address', 'bin', 'bbl', 'state', 'problems','status','stump_diam'])

In [7]:
from features import Features

features = Features()
num_features = features.get_num_features()
cat_features = features.get_cat_features()

# num_features = ['tree_dbh']  # Только tree_dbh из числовых
# cat_features = ['spc_latin', 'spc_common', 'postcode', 'borough', 'zip_city', 'steward', 'guards', 'sidewalk', 'user_type',
#                 'root_stone', 'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other', 'brch_light', 'brch_shoe', 'brch_other','curb_loc']

In [8]:
pd.set_option('display.max_columns', None)
df_cleaned

,tree_dbh,curb_loc,health,spc_latin,spc_common,steward,guards,sidewalk,user_type,root_stone,root_grate,root_other,trunk_wire,trnk_light,trnk_other,brch_light,brch_shoe,brch_other,postcode,zip_city,community board,borocode,borough,cncldist,st_assem,st_senate,council district,census tract
0,3,OnCurb,Fair,Acer rubrum,red maple,None,None,NoDamage,TreesCount Staff,No,No,No,No,No,No,No,No,No,11375,Forest Hills,406,4,Queens,29,28,16,29.0,739.0
1,9,OnCurb,Fair,Acer platanoides,Norway maple,None,None,Damage,Volunteer,Yes,No,No,Yes,No,Yes,Yes,No,No,11355,Flushing,407,4,Queens,20,40,16,20.0,855.0
2,24,OnCurb,Good,Acer platanoides,Norway maple,None,None,NoDamage,Volunteer,No,No,No,No,No,No,No,No,No,11375,Forest Hills,406,4,Queens,29,27,14,29.0,739.0
3,2,OnCurb,Good,Juglans nigra,black walnut,None,None,NoDamage,Volunteer,No,No,Yes,No,No,No,No,No,No,11217,Brooklyn,306,3,Brooklyn,33,52,25,33.0,71.0
4,2,OnCurb,Poor,Koelreuteria paniculata,golden raintree,1or2,None,NoDamage,TreesCount Staff,No,No,No,No,No,No,No,No,No,11694,Rockaway Park,414,4,Queens,32,23,15,32.0,928.0
5,3,OnCurb,Poor,Acer platanoides 'Crimson King',crimson king maple,None,None,NoDamage,TreesCount Staff,No,No,No,No,No,No,No,No,No,10308,Staten Island,503,5,Staten Island,51,62,24,51.0,14607.0


In [9]:
import joblib

# Загружаем LabelEncoder для целевой переменной
label_encoder = joblib.load('label_encoder_health.pkl')

# Загружаем LabelEncoder'ы для категориальных признаков
label_encoders = {}
for col in cat_features:
    label_encoders[col] = joblib.load(f'label_encoder_{col}.pkl')

In [19]:
df_cleaned

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,steward,guards,sidewalk,user_type,problems,root_stone,root_grate,root_other,trunk_wire,trnk_light,trnk_other,brch_light,brch_shoe,brch_other,address,postcode,zip_city,community board,borocode,borough,cncldist,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl
0,180683,348711,2015-08-27T00:00:00.000,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,NaN,NaN,NoDamage,TreesCount Staff,NaN,No,No,No,No,No,No,No,No,No,108-005 70 AVENUE,11375,Forest Hills,406,4,Queens,29,28,16,QN17,Forest Hills,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,169659,312096,2015-08-23T00:00:00.000,9,0,OnCurb,Alive,Fair,Acer platanoides,Norway maple,NaN,NaN,Damage,Volunteer,"Stones,WiresRope,TrunkOther,BranchLights",Yes,No,No,Yes,No,Yes,Yes,No,No,143-016 41 AVENUE,11355,Flushing,407,4,Queens,20,40,16,QN22,Flushing,4085500,New York,40.759994,-73.823264,1.033211e+06,216212.4190,20.0,855.0,4443459.0,4.050480e+09
2,165478,348286,2015-08-22T00:00:00.000,24,0,OnCurb,Alive,Good,Acer platanoides,Norway maple,NaN,NaN,NoDamage,Volunteer,NaN,No,No,No,No,No,No,No,No,No,76-028 GRAND CENTRAL PARKWAY SR WEST,11375,Forest Hills,406,4,Queens,29,27,14,QN17,Forest Hills,4075701,New York,40.717622,-73.832601,1.030654e+06,200769.7983,29.0,739.0,NaN,NaN
3,150316,224157,2015-08-16T00:00:00.000,2,0,OnCurb,Alive,Good,Juglans nigra,black walnut,NaN,NaN,NoDamage,Volunteer,RootOther,No,No,Yes,No,No,No,No,No,No,226 NEVINS STREET,11217,Brooklyn,306,3,Brooklyn,33,52,25,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,3007100,New York,40.681757,-73.985852,9.881740e+05,187659.2399,33.0,71.0,3336089.0,3.004110e+09
4,159168,337886,2015-08-19T00:00:00.000,2,0,OnCurb,Alive,Poor,Koelreuteria paniculata,golden raintree,1or2,NaN,NoDamage,TreesCount Staff,NaN,No,No,No,No,No,No,No,No,No,139-015 ROCKAWAY BEACH BOULEVARD,11694,Rockaway Park,414,4,Queens,32,23,15,QN10,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,4092800,New York,40.572399,-73.856627,1.024081e+06,147849.5452,32.0,928.0,4306405.0,4.162840e+09
5,194731,413529,2015-09-01T00:00:00.000,3,0,OnCurb,Alive,Poor,Acer platanoides 'Crimson King',crimson king maple,NaN,NaN,NoDamage,TreesCount Staff,NaN,No,No,No,No,No,No,No,No,No,94 EAST BRANDIS AVENUE,10308,Staten Island,503,5,Staten Island,51,62,24,SI54,Great Kills,5014607,New York,40.559243,-74.163276,9.388810e+05,143066.1083,51.0,14607.0,5073103.0,5.054940e+09


In [10]:
# Применяем LabelEncoder'ы к категориальным признакам
for col in cat_features:
    df_cleaned[col] = label_encoders[col].transform(df_cleaned[col])

# Теперь new_data готов для передачи в модель

In [11]:
#F F G G P P
X = df_cleaned[num_features + cat_features]
X

,tree_dbh,spc_latin,spc_common,postcode,borough,zip_city,steward,guards,sidewalk,user_type,root_stone,root_grate,root_other,trunk_wire,trnk_light,trnk_other,brch_light,brch_shoe,brch_other,curb_loc
0,3,11,103,157,3,16,3,2,1,1,0,0,0,0,0,0,0,0,0,1
1,9,8,32,138,3,15,3,2,0,2,1,0,0,1,0,1,1,0,0,1
2,24,8,32,157,3,16,3,2,1,2,0,0,0,0,0,0,0,0,0,1
3,2,61,54,113,1,6,3,2,1,2,0,0,1,0,0,0,0,0,0,1
4,2,63,76,189,3,37,0,2,1,1,0,0,0,0,0,0,0,0,0,1
5,3,9,65,56,4,43,3,2,1,1,0,0,0,0,0,0,0,0,0,1


In [22]:
# import torch.nn as nn

# num_classes = 3

# # Модель
# class TreeHealthModel(nn.Module):
#     def __init__(self, num_numeric, cat_cardinalities, embedding_dim=10, hidden_dim=128):
#         super(TreeHealthModel, self).__init__()
        
#         # Эмбеддинги для категориальных признаков
#         self.embeddings = nn.ModuleList([nn.Embedding(cat_size, embedding_dim) for cat_size in cat_cardinalities])
        
#         # Размер входных данных
#         input_size = num_numeric + len(cat_cardinalities) * embedding_dim
        
#         # Полносвязные слои
#         self.fc1 = nn.Linear(input_size, hidden_dim)
#         self.fc2 = nn.Linear(hidden_dim, hidden_dim)
#         self.fc3 = nn.Linear(hidden_dim, num_classes)
        
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(0.3)
    
#     def forward(self, x_num, x_cat):
#         cat_embeddings = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
#         cat_embeddings = torch.cat(cat_embeddings, dim=1)
#         x = torch.cat([x_num, cat_embeddings], dim=1)
#         x = self.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [12]:
import torch
import joblib
from model import TreeHealthModel

# Загружаем cat_cardinalities
cat_cardinalities = joblib.load('cat_cardinalities.pkl')

# Загрузка модели
model = TreeHealthModel(num_numeric=len(num_features), cat_cardinalities=cat_cardinalities, embedding_dim=20, hidden_dim=256)
model.load_state_dict(torch.load('tree_health_model.pth'))
model.eval()  # Переводим модель в режим оценки

/var/folders/v3/v5xg2g_x3179bvpwbk508_jw0000gn/T/ipykernel_25029/2277899825.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('tree_healt

TreeHealthModel(
  (embeddings): ModuleList(
    (0-1): 2 x Embedding(132, 20)
    (2): Embedding(191, 20)
    (3): Embedding(5, 20)
    (4): Embedding(48, 20)
    (5-6): 2 x Embedding(4, 20)
    (7): Embedding(2, 20)
    (8): Embedding(3, 20)
    (9-18): 10 x Embedding(2, 20)
  )
  (fc1): Linear(in_features=381, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=3, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
)

In [15]:
import torch

# Предположим, у вас есть датафрейм X с 10 примерами
# Преобразуем числовые и категориальные признаки в тензоры
X_num = torch.tensor(X[num_features].values, dtype=torch.float32)  # Числовые признаки
X_cat = torch.tensor(X[cat_features].values, dtype=torch.long)    # Категориальные признаки

# Переводим модель в режим оценки
model.eval()

# Получаем предсказания
with torch.no_grad():  # Отключаем вычисление градиентов
    outputs = model(X_num, X_cat)  # Передаем данные в модель
    _, predicted = torch.max(outputs, 1)  # Преобразуем выходы в классы
    predictions = predicted.cpu().numpy()  # Сохраняем предсказания

# Декодируем предсказания (если целевая переменная была закодирована)
predicted_labels = label_encoder.inverse_transform(predictions)

# Добавляем предсказания в датафрейм
X['predicted_health'] = predicted_labels

# Выводим результат
print(X[['tree_dbh', 'spc_latin', 'predicted_health']])

   tree_dbh  spc_latin predicted_health
0         3         11             Poor
1         9          8             Fair
2        24          8             Fair
3         2         61             Good
4         2         63             Fair
5         3          9             Poor


/var/folders/v3/v5xg2g_x3179bvpwbk508_jw0000gn/T/ipykernel_25029/291679368.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['predicted_health'] = predicted_labels
